<a href="https://colab.research.google.com/github/sheidaaa-sh/shoooody/blob/main/Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving 20240516_training_data (1).xlsx to 20240516_training_data (1).xlsx


In [2]:
import pandas as pd

# Load the dataset from an Excel file
df = pd.read_excel('/content/20240516_training_data (1).xlsx')

# Display the first few rows to verify
print(df.head())

   PRENOTAZIONE COD_AZIENDA                     Data  FLAG_FATTURATO  \
0        818020         BeT  2023-01-02 00:00:00.000               1   
1        818021         BeT  2023-01-02 00:00:00.000               1   
2        818022         BeT  2023-01-02 00:00:00.000               1   
3        818023         BeT  2023-01-02 00:00:00.000               1   
4        818024         BeT  2023-01-02 00:00:00.000               1   

   id_quotazioni  id_quotazioni_servizio  id_quotazioni_servizio_acquisti  \
0       15267116                21318146                              NaN   
1       15276481                21451129                              NaN   
2       15276450                21450651                              NaN   
3       15267123                21318250                              NaN   
4       15267129                21318330                              NaN   

  desc_tipo_servizio Result_Account Result_Supplier  ...  \
0        B&T Express      106309247         

# Handling missing value

nothing was neccesory

# Feature encoding

Categorical encoding:

In [4]:
df = pd.get_dummies(df, columns=['service_type', 'shipping_type', 'goods_type'])

Text features:

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('italian'))

# Define a function to clean text
def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]  # Lemmatize and remove stopwords
    return " ".join(text)

# Apply to both columns
df['Good_description_clean'] = df['goods_description'].apply(lambda x: clean_text(str(x)))
df['Observations_clean'] = df['observations'].apply(lambda x: clean_text(str(x)))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
print(df.head())

   PRENOTAZIONE COD_AZIENDA                     Data  FLAG_FATTURATO  \
0        818020         BeT  2023-01-02 00:00:00.000               1   
1        818021         BeT  2023-01-02 00:00:00.000               1   
2        818022         BeT  2023-01-02 00:00:00.000               1   
3        818023         BeT  2023-01-02 00:00:00.000               1   
4        818024         BeT  2023-01-02 00:00:00.000               1   

   id_quotazioni  id_quotazioni_servizio  id_quotazioni_servizio_acquisti  \
0       15267116                21318146                              NaN   
1       15276481                21451129                              NaN   
2       15276450                21450651                              NaN   
3       15267123                21318250                              NaN   
4       15267129                21318330                              NaN   

  desc_tipo_servizio Result_Account Result_Supplier  ...  \
0        B&T Express      106309247         

# Option 1: Word embedding

In [ ]:
import numpy as np
import gensim.downloader as api

# Load pre-trained Word2Vec embeddings
word2vec_model = api.load("word2vec-google-news-300")

# Function to compute the average word vectors for a document
def document_vector(doc):
    doc = [word for word in doc.split() if word in word2vec_model]
    if len(doc) > 0:
        return np.mean(word2vec_model[doc], axis=0)
    else:
        return np.zeros(300)  # Handle empty documents

# Apply to the dataset
df['Good_description_vector'] = df['Good_description_clean'].apply(lambda x: document_vector(x))
df['Observations_vector'] = df['Observations_clean'].apply(lambda x: document_vector(x))


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# Check the structure of one of the new columns
print(df['Good_description_vector'].head())
print(df['Observations_vector'].head())

0    [-0.020095825, -0.035003662, -0.024627686, 0.1...
1    [-0.031194052, -0.06982422, 0.023803711, -0.02...
2    [0.025390625, 0.03491211, -0.09150696, 0.25292...
3    [0.03930664, 0.014526367, 0.06298828, -0.01019...
4    [0.03930664, 0.014526367, 0.06298828, -0.01019...
Name: Good_description_vector, dtype: object
0    [-0.29492188, 0.075683594, 0.1640625, 0.142578...
1    [-0.29492188, 0.075683594, 0.1640625, 0.142578...
2    [-0.29492188, 0.075683594, 0.1640625, 0.142578...
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4    [-0.29492188, 0.075683594, 0.1640625, 0.142578...
Name: Observations_vector, dtype: object


In [ ]:
# Expand the 'Good_description_vector' into 300 separate columns
good_desc_vector_df = pd.DataFrame(df['Good_description_vector'].tolist(), index=df.index)
good_desc_vector_df.columns = [f'Good_desc_vec_{i}' for i in range(300)]

# Expand the 'Observations_vector' into 300 separate columns
observations_vector_df = pd.DataFrame(df['Observations_vector'].tolist(), index=df.index)
observations_vector_df.columns = [f'Obs_vec_{i}' for i in range(300)]

In [ ]:
# Drop original text and vector columns
df = df.drop(columns=['goods_description', 'observations', 'Good_description_clean', 'Observations_clean', 'Good_description_vector', 'Observations_vector'])


In [ ]:
# Combine the original DataFrame with the new vector DataFrames
df_final = pd.concat([df, good_desc_vector_df, observations_vector_df], axis=1)


In [ ]:
print(df_final.head())

print(df_final.shape)

   PRENOTAZIONE COD_AZIENDA                     Data  FLAG_FATTURATO  \
0        818020         BeT  2023-01-02 00:00:00.000               1   
1        818021         BeT  2023-01-02 00:00:00.000               1   
2        818022         BeT  2023-01-02 00:00:00.000               1   
3        818023         BeT  2023-01-02 00:00:00.000               1   
4        818024         BeT  2023-01-02 00:00:00.000               1   

   id_quotazioni  id_quotazioni_servizio  id_quotazioni_servizio_acquisti  \
0       15267116                21318146                              NaN   
1       15276481                21451129                              NaN   
2       15276450                21450651                              NaN   
3       15267123                21318250                              NaN   
4       15267129                21318330                              NaN   

  desc_tipo_servizio Result_Account Result_Supplier  ... Obs_vec_290  \
0        B&T Express      106309

In [ ]:
df_final.to_csv('combined_dataset.csv', index=False)

# Option 2: TF-idf ==> better

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
tfidf_good_desc = TfidfVectorizer(max_features=100)
tfidf_observations = TfidfVectorizer(max_features=100)

# Fit and transform the data
good_desc_tfidf = tfidf_good_desc.fit_transform(df['Good_description_clean'])
observations_tfidf = tfidf_observations.fit_transform(df['Observations_clean'])

# Convert to DataFrame
good_desc_tfidf_df = pd.DataFrame(good_desc_tfidf.toarray(), columns=tfidf_good_desc.get_feature_names_out())
observations_tfidf_df = pd.DataFrame(observations_tfidf.toarray(), columns=tfidf_observations.get_feature_names_out())

# Concatenate with the original DataFrame
df = pd.concat([df, good_desc_tfidf_df, observations_tfidf_df], axis=1)


In [ ]:
print(df.head(10))

   PRENOTAZIONE COD_AZIENDA                     Data  FLAG_FATTURATO  \
0        818020         BeT  2023-01-02 00:00:00.000               1   
1        818021         BeT  2023-01-02 00:00:00.000               1   
2        818022         BeT  2023-01-02 00:00:00.000               1   
3        818023         BeT  2023-01-02 00:00:00.000               1   
4        818024         BeT  2023-01-02 00:00:00.000               1   
5        818025         BeT  2023-01-02 00:00:00.000               1   
6        818027         BeT  2023-01-02 00:00:00.000               1   
7        818028         BeT  2023-01-02 00:00:00.000               1   
8        818029         BeT  2023-01-02 00:00:00.000               1   
9        818030         BeT  2023-01-02 00:00:00.000               1   

   id_quotazioni  id_quotazioni_servizio  id_quotazioni_servizio_acquisti  \
0       15267116                21318146                              NaN   
1       15276481                21451129             

In [ ]:
# Combine all encoded features with the main dataset
df_final = pd.concat([df.drop(columns=['goods_description', 'observations']), good_desc_tfidf_df, observations_tfidf_df], axis=1)


In [ ]:
print(df_final.head())
print(df_final.columns)

   PRENOTAZIONE COD_AZIENDA                     Data  FLAG_FATTURATO  \
0        818020         BeT  2023-01-02 00:00:00.000               1   
1        818021         BeT  2023-01-02 00:00:00.000               1   
2        818022         BeT  2023-01-02 00:00:00.000               1   
3        818023         BeT  2023-01-02 00:00:00.000               1   
4        818024         BeT  2023-01-02 00:00:00.000               1   

   id_quotazioni  id_quotazioni_servizio  id_quotazioni_servizio_acquisti  \
0       15267116                21318146                              NaN   
1       15276481                21451129                              NaN   
2       15276450                21450651                              NaN   
3       15267123                21318250                              NaN   
4       15267129                21318330                              NaN   

  desc_tipo_servizio Result_Account Result_Supplier  ... unarea under usare  \
0        B&T Express     